In [2]:
import logging
from os import chdir
import pandas as pd

from unicef_schools_attribute_cleaning.utils.standardize_column_names import standardize_column_names
from unicef_schools_attribute_cleaning.models.school_pandas_filter import school_pandas_filter

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# df = pd.read_excel('./data/UNICE_schools_raw_2020_Jun/Zimbabwe schools complete data.xlsx')
src_df = pd.read_csv('../../data/unicef_fixed/zimbabwe_original_uuid.csv', low_memory=False)

# apply country code, provider to every row
src_df['country_code'] = 'ZW'
src_df['is_private'] = True
src_df['provider'] = 'devseed'
src_df['provider_is_private'] = True

# logger.info(pp(df.columns))

tmp = standardize_10column_names(src_df)

# logger.info(pp(df))

# optional speedup: we know the lat,lon columns are numeric AND are required, so preprocess with pandas
# this gets rid of rows with non-numeric lat/long '(blank)' etc.
tmp = tmp[pd.to_numeric(tmp['lat'], errors='coerce').notnull()]
tmp = tmp[pd.to_numeric(tmp['lon'], errors='coerce').notnull()]


head = tmp.head(1000)
tmp = head.apply(func=school_pandas_filter, axis=1)

# tmp = tmp.apply(func=school_pandas_filter, axis=1)

# filter out the None values from the schools_pandas_filter apply

cleaned_schools_df = tmp[tmp['uuid'].notnull()]

logger.info(f'{len(src_df)} source schools, {len(cleaned_schools_df)} cleaned schools')
logger.info(cleaned_schools_df.head()[['name', 'num_students', 'type_connectivity', 'educ_level']])

logger.info(cleaned_schools_df['num_students'])

INFO:root:renaming columns: {'Enrolment': ('num_students', 'fuzzy match on alias column: enrollment 95%'),
 'Name': ('name', 'fuzzy match 100%'),
 'Province': ('admin2', 'alias exact match'),
 'SchoolLevel': ('educ_level', 'fuzzy match on alias column: school level 96%'),
 'Schoolnumber': ('description',
                  'fuzzy match on alias column: school number 96%'),
 '_tot_func_computers': ('num_computers',
                         'fuzzy match on alias column: tot_func_computers 97%'),
 'connectiontype': ('type_connectivity', 'alias exact match'),
 'latitude': ('lat', 'alias exact match'),
 'longitude': ('lon', 'alias exact match')}
INFO:root:adding 34 columns from schema: ['admin0',
 'admin1',
 'admin3',
 'admin4',
 'admin_code',
 'admin_id',
 'address',
 'address2',
 'phone_number',
 'person_contact',
 'email',
 'postal_code',
 'altitude',
 'gps_confidence',
 'date',
 'num_teachers',
 'connectivity',
 'speed_connectivity',
 'latency_connectivity',
 'availability_connectivity',

KeyError: 'uuid'